In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# データ読み込み
notepc_data = pickle.load(open("notepc_data.pkl", "br"))

# カラム
# 0: price
# 1: num_of_cores
# 2: clock
# 3: memory
# 4: disk
# 5: weight
# 6: months

In [ ]:
# 実際に使うデータを切り出し
# 価格は 1/100000スケールにて。価格もクロック数も、（ベクトルのままではなく）行列形式に変換
price = np.array([t[0] for t in notepc_data], dtype=np.float32) / 100000.
price.resize((price.size, 1))
clock = np.array([t[2] for t in notepc_data], dtype=np.float32)
clock.resize((clock.size, 1))

In [ ]:
clock.shape

In [ ]:
# データ可視化
plt.scatter(clock, price)
plt.xlabel("clock")
plt.ylabel("price")

In [ ]:
# パラメータ、データ、モデルを用意

# パラメータ初期値
w1_ar = np.array([[-0.683, -0.833, -0.049]], dtype=np.float32)
b1_ar = np.array([0.499, 0.085, -0.654], dtype=np.float32)
w2_ar = np.array([[246071, 125136, 36801]], dtype=np.float32) / 100000.
b2_ar = np.array([116149], dtype=np.float32) / 100000.

# 上記の値を使い、variable定義
w1 = tf.Variable(w1_ar, dtype=tf.float32)
b1 = tf.Variable(b1_ar, dtype=tf.float32)
w2 = tf.Variable(w2_ar, dtype=tf.float32)
b2 = tf.Variable(b2_ar, dtype=tf.float32)

# データの入り口、placeholderを x (clock)、t (price) について定義
x_ph = tf.placeholder(tf.float32)
t_ph = tf.placeholder(tf.float32)


# == モデル定義 ==
# 1層目
h1 = tf.nn.sigmoid(x_ph * w1 + b1)
    
# 2層目
y = tf.matmul(h1, tf.transpose(w2)) + b2

# == コスト定義 ==
cost = 0.5 * tf.reduce_mean((y - t_ph) ** 2.)

# == 学習用のopを定義 ==
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.3)
train_op = trainer.minimize(cost, var_list=[w1, b1, w2, b2])

In [ ]:
# モデルのプロット用関数
x_val = np.linspace(1., 3.2, 101)
x_val.resize((x_val.size, 1))

def plot(scale=1.):        
    y_val = sess.run(y, feed_dict={x_ph: x_val})
    plt.plot(x_val, y_val * scale, color="orange")

    plt.scatter(clock, price * scale)
    plt.xlabel("clock")
    plt.ylabel("price")
    plt.ylim([(price *scale).min() - 0.15 * scale, (price * scale).max() + 0.15 * scale])

In [ ]:
# モデル可視化

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    plot();

In [ ]:
def model_save(sess):
    return sess.run([w1, b1, w2, b2])

In [ ]:
assigner = tf.placeholder(tf.float32)

def load_model(sess, param_list):
    sess.run(tf.assign(w1, assigner), feed_dict={assigner: param_list[0]})
    sess.run(tf.assign(b1, assigner), feed_dict={assigner: param_list[1]})
    sess.run(tf.assign(w2, assigner), feed_dict={assigner: param_list[2]})
    sess.run(tf.assign(b2, assigner), feed_dict={assigner: param_list[3]})

In [ ]:
# 実際の計算
history = list()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        # 現状のコスト値を計算、記録
        fd = {x_ph: clock, t_ph: price}
        cost_val = sess.run(cost, feed_dict=fd)
        history.append(cost_val)

        # 進捗モニタリング
        if i % 1000 == 0:
            print("{:>6d}, cost: {:7.5f}".format(i, cost_val))

        # 学習
        sess.run(train_op, feed_dict=fd)
        

        # ===== original code =====
        # 学習

        ## == 2層目 ==
        ## errorを出しておく
        # e2 = y - price

        ## jを w2、bで微分したものをそれぞれ求める
        # j_w2_deriv = e2 * h1
        # j_b2_deriv = e2

        ## 新しいパラメータ w2_new、b2_newを計算
        # w2_new = w2 - alpha * np.mean(j_w2_deriv, axis=0, keepdims=True)
        # b2_new = b2 - alpha * np.mean(j_b2_deriv, axis=0)

        ## == 1層目 ==
        ## errorを出しておく
        # e1 = e2 * w2 * h1 * (1. - h1)

        ## jを w1、b1で微分したものをそれぞれ求める
        # j_w1_deriv = e1 * clock
        # j_b1_deriv = e1

        ## 新しいパラメータ w1_new、b1_newを計算
        # w1_new = w1 - alpha * np.mean(j_w1_deriv, axis=0, keepdims=True)
        # b1_new = b1 - alpha * np.mean(j_b1_deriv, axis=0)

        ## 新旧パラメータをまとめて入れ替え
        # w2 = w2_new
        # b2 = b2_new
        # w1 = w1_new
        # b1 = b1_new
        
        
    # 結果として得られたモデルの可視化
    plot()
    
    # 結果として得られたモデルの保存
    param_list = model_save(sess)

In [ ]:
# 学習曲線
plt.plot([t for t in history])
plt.xlabel("iterations")
plt.ylabel("cost")

In [ ]:
# clock数に対応すうる yの値を全て表示するには？

with tf.Session() as sess:
    load_model(sess, param_list)
    
    # yの値の計算
    
    # 表示
    